In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

from sklearn.model_selection import (train_test_split,
                                     GridSearchCV,
                                     StratifiedKFold,
                                     StratifiedGroupKFold)

from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score,
                             f1_score,
                             balanced_accuracy_score,
                             roc_auc_score)

from sklearn.metrics import make_scorer

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from pprint import pprint

In [2]:
random_state = 0
spec_threshold = 0.90

DATA_PATH = ""
export_results_path = ""
export_raw_final_analysis_group_path = ""
export_raw_predictions_path = ""


In [3]:
categorical_cols = [
        '1.7 Gender',
        '1.8 Race',
        '1.9 Ethnicity',
        '2.3 Personal_history of HCC',
        '2.4 Personal History of treated HCC',
        '2.5 Liver Disease Etiology (choice=Alcohol)',
        '2.5 Liver Disease Etiology (choice=NASH)',
        '2.5 Liver Disease Etiology (choice=Hepatitis B)',
        '2.5 Liver Disease Etiology (choice=Hepatitis C)',
        '2.5 Liver Disease Etiology (choice=Other)',
        '2.7 Cirrhosis',
        '2.11 Encephalopathy',
        '2.12 Ascites',
        '2.15 Child Pugh Score',
        '6.7 Location (Couinaud segment)',
        '6.9 Nodule Echogenicity',
        '6.10 Nodule Margin',
        '6.17 Arterial Phase Enhancement',
        '6.18 Arterial Phase Enhancement Distribution',
        '6.19 Washout',
        '6.21 Washout Degree',
        '6.25 CEUS LI-RADS'
    ]

confirmatory_tests_cols = [
    '7.1 Date',
    '7.2 Type of examination',
    '7.3 Total number of focal liver lesions',
    '7.4 Nodule',
    '7.5 Nodule Longest Diameter, mm',
    '7.6 Location (Couinaud segment)',
    '7.7 Arterial Phase Enhancement',
    '7.8 Washout',
    '7.9 Portal Vein Thrombosis',
    '7.10 Tumor Capsule',
    '7.11 Threshold Growth',
    '7.12 Ancillary Features',
    '7.13 Features that may favor malignancy (choice=Hepatobiliary phase hypointensity)',
    '7.13 Features that may favor malignancy (choice=Transitional phase hypointensity)',
    '7.13 Features that may favor malignancy (choice=Mild-moderate T2 hyperintensity)',
    '7.13 Features that may favor malignancy (choice=Restricted diffusion)',
    '7.13 Features that may favor malignancy (choice=Distinctive rim*)',
    '7.13 Features that may favor malignancy (choice=Corona enhancement*)',
    '7.13 Features that may favor malignancy (choice=Mosaic architecture*)',
    '7.13 Features that may favor malignancy (choice=Nodule-in-nodule architecture*)',
    '7.13 Features that may favor malignancy (choice=Intra-lesional fat*)',
    '7.13 Features that may favor malignancy (choice=Lesional iron sparing)',
    '7.13 Features that may favor malignancy (choice=Lesional fat sparing)',
    '7.13 Features that may favor malignancy (choice=Blood products)',
    '7.13 Features that may favor malignancy (choice=Diameter increase less than threshold growth)',
    '7.14 Features that may favor benignity (choice=Homogeneous marked T2 hyperintensity)',
    '7.14 Features that may favor benignity (choice=Homogeneous marked T2 or T2* hypointensity)',
    '7.14 Features that may favor benignity (choice=Hepatobiliary phase isointensity)',
    '7.14 Features that may favor benignity (choice=Undistorted vessels)',
    '7.14 Features that may favor benignity (choice=Parallels blood pool enhancement)',
    '7.14 Features that may favor benignity (choice=Diameter reduction)',
    '7.14 Features that may favor benignity (choice=Diameter stability ≥ 2 years)',
    '7.15 Tie-breaking Rules Applied',
    '7.16 CT / MRI LI-RADS Classification',

    '8.1 Date',
    '8.2 Nodule',
    '8.3 Nodule Longest Diameter, mm',
    '8.4 Location (Couinaud segment)',
    '8.5 Nodule Tissue Sampling',
    '8.6 Needle Size (G)',
    '8.7 Results - Tumor Type',
    '8.8 Specify',
    '8.9 Edmondson and Steiner Grade',
    '8.10 WHO Grade',
    '8.10 Adjacent Liver Tissue Sampling',
    '8.11 Needle Size (G)',
    '8.12 Results',
    '8.13 Specify',

    'Nodule',
    'Final diagnosis source',
    'FINAL DIAGNOSIS',
    'Final notes'
]

# functions

In [4]:
def remove_preceding_nans(row):
    # read from left to right
    for i in range(len(row)):
        if not pd.isnull(row.iloc[i]):
            return row[i:]

def remove_terminating_nans(row):
    # read from right to left
    for i in range(len(row)-1, -1, -1):
        if not pd.isnull(row.iloc[i]):
            return row[:i+1]

def extract_year_from_str_date(x):
    # yyyy-mm-dd
    return int(x.split('-')[0])

def map_segment_to_str(x):
    return 'segment ' + str(int(x))


def convert_categorical_to_onehot(df):
    # convert categorical columns to one-hot encoding
    df = df.copy()
    for col in categorical_cols:
        df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
        df = df.drop(col, axis=1)
    return df


def descriptive_analyses(df):
    for group in [['train','test'], ['train'], ['test']]:
        print('Group:', group)
        print('Number of patients:', df.loc[df['allocated_set'].isin(group)]['1.1 Patient ID'].nunique())
        print('Number of nodules:', df.loc[df['allocated_set'].isin(group)].shape[0])
        
        # categorical columns
        for col in categorical_cols + ['GT_benign(0)_malignant(1)','7.16 CT / MRI LI-RADS Classification','Final diagnosis source']:
            print(group)
            if col == '7.16 CT / MRI LI-RADS Classification':
                print(col)
                print(df.loc[df['allocated_set'].isin(group) & df['Final diagnosis source'].isin(['Initial CT/MRI','Follow-up CT/MRI'])][col].value_counts(dropna=False))
                print('\n')
            else:
                print(col)
                print(df.loc[df['allocated_set'].isin(group)][col].value_counts(dropna=False))
                print('\n')

        # continuous columns
        for col in df.columns:
            if col not in categorical_cols and col not in ['1.1 Patient ID','allocated_set']+confirmatory_tests_cols:
                print(group)
                if col == '2.14 MELD Score':
                    print(col)
                    print(df.loc[(df['allocated_set'].isin(group)) & (df['2.14 MELD Score']!=0)][col].astype('float').describe())
                    print('\n')
                elif col == '6.20 Timing of Washout Onset, sec':
                    print(col)
                    print(df.loc[(df['allocated_set'].isin(group)) & (df['6.20 Timing of Washout Onset, sec']!=5000)][col].astype('float').describe())
                    print('\n')
                else:
                    print(col)
                    print(df.loc[df['allocated_set'].isin(group)][col].astype('float').describe())
                    print('\n')

def load_data(data_path):
    df = pd.read_csv(data_path)
    df.loc[:,'Repeat Instrument'] = df['Repeat Instrument'].fillna(value='demographics_and_labs')
    df.loc[:,'Repeat Instance'] = df['Repeat Instance'].fillna(value=1)
    df.columns = df.columns.str.strip()

    map_columns = {
        'demographics_and_labs':[
            '1.2 Site ID',
            '1.3 Patient Screening Date',
            '1.4 Patient Consent Date',
            '1.5 Patient Initials',
            '1.6 Year of Birth',
            '1.7 Gender',
            '1.8 Race',
            '1.9 Ethnicity',
            '1.10 Zip Code (only required for US sites)',
            '1.11 Inclusion criteria (choice=Patients ≥21 years of age.)',
            '1.11 Inclusion criteria (choice=Capable of making informed decisions regarding his/her treatment.)',
            '1.11 Inclusion criteria (choice=Have known cirrhosis or other risk factors for HCC, based on AASLD and EASL guidelines.)',
            '1.11 Inclusion criteria (choice=Patients with untreated focal liver observations on liver ultrasound or multiphase contrast-enhanced CT or MRI performed as part of clinical standard of care within 4 weeks before patient enrollment.)',
            '1.11 Inclusion criteria (choice=Patients with untreated focal liver observations scheduled for follow-up multiphase contrast-enhanced CT or MRI, biopsy or surgical excision as part of clinical standard of care. CEUS should be performed within 4 weeks before or after follow-up imaging or within 4 weeks before biopsy or surgical excision.)',
            '1.12 Exclusion Criteria (choice=Patients who are pregnant or lactating.)',
            '1.12 Exclusion Criteria (choice=Patients with focal liver observations less than 5 mm or greater than 5 cm in size.)',
            '1.12 Exclusion Criteria (choice=Patients with contraindications to CEUS.)',
            '1.12 Exclusion Criteria (choice=Patients with contraindications to both CT and MRI.)',
            '1.12 Exclusion Criteria (choice=Patients who are medically unstable, terminally ill, or whose clinical course is unpredictable.)',
            '1.12 Exclusion Criteria (choice=Liver nodule previously treated with trans-arterial or thermal ablation.)',
            '1.12 Exclusion Criteria (choice=Patients who have received an investigational drug in the 30 days before CEUS, or will receive one within 72 hour after their CEUS exam.)',
            '2.1 Weight (kilograms)',
            '2.2 Height (cm)',
            '2.3 Personal_history of HCC',
            '2.4 Personal History of treated HCC',
            '2.5 Liver Disease Etiology (choice=Alcohol)',
            '2.5 Liver Disease Etiology (choice=NASH)',
            '2.5 Liver Disease Etiology (choice=Hepatitis B)',
            '2.5 Liver Disease Etiology (choice=Hepatitis C)',
            '2.5 Liver Disease Etiology (choice=Other)',
            '2.5 Liver Disease Etiology (choice=N/A)',
            '2.6 Specify',
            '2.7 Cirrhosis',
            '2.8 Cirrhosis Biopsy Proven',
            '2.9 Biopsy Date',
            '2.10 Dialysis at least twice in the past week',
            '2.11 Encephalopathy',
            '2.12 Ascites',
            '2.13 ECOG performance status',
            '2.14 MELD Score',
            '2.15 Child Pugh Score',
            '2.16 BCLC Stage',
            '5.1 Albumin, g/dL',
            '5.2 AFP, ng/ml',
            '5.3 ALT, units',
            '5.4 AST, units',
            '5.5 Bilirubin (Total), mg/dL',
            '5.6 Creatinine, mg/dL',
            '5.7 Sodium, mEq/L',
            '5.8 INR'
        ],
        'F3: Ultrasound Imaging':[
            '3.1 Date',
            '3.2 Sonographer / Sonologist Initials',
            '3.3 Years of CEUS Experience',
            '3.4 Dose of Contrast (ml)   Recommended dose 2.4ml',
            '3.5 Second dose, if performed',
            '3.6 Injection Site',
            '3.7 Vein',
            '3.8 Peripheral vein catheter size',
            '3.9 Name of person who injected Lumason'
        ],
        'F6: Ultrasound Imaging Analysis':[
            '6.1 Date',
            '6.2 Nodule',
            '6.3 Pre-contrast US Exam',
            '6.4 Specify',
            '6.5 Nodule Visible on pre-contrast US',
            '6.6 Nodule Longest Diameter, mm',
            '6.7 Location (Couinaud segment)',
            '6.8 Nodule Composition',
            '6.9 Nodule Echogenicity',
            '6.10 Nodule Margin',
            '6.11 Liver SWI Performed',
            '6.12 SWI Scanner',
            '6.13 Liver SWI Results, kPa',
            '6.14 Liver SWI Results, m/sec',
            '6.15 CEUS Exam',
            '6.16 Specify',
            '6.17 Arterial Phase Enhancement',
            '6.18 Arterial Phase Enhancement Distribution',
            '6.19 Washout',
            '6.20 Timing of Washout Onset, sec',
            '6.21 Washout Degree',
            '6.22 Tie-breaking Rules Applied',
            '6.23 Ancillary Features Applied',
            '6.24 Specify',
            '6.25 CEUS LI-RADS'
        ],
        'F7: CT or MRI Imaging Analysis':[
            '7.1 Date',
            '7.2 Type of examination',
            '7.3 Total number of focal liver lesions',
            '7.4 Nodule',
            '7.5 Nodule Longest Diameter, mm',
            '7.6 Location (Couinaud segment)',
            '7.7 Arterial Phase Enhancement',
            '7.8 Washout',
            '7.9 Portal Vein Thrombosis',
            '7.10 Tumor Capsule',
            '7.11 Threshold Growth',
            '7.12 Ancillary Features',
            '7.13 Features that may favor malignancy (choice=Hepatobiliary phase hypointensity)',
            '7.13 Features that may favor malignancy (choice=Transitional phase hypointensity)',
            '7.13 Features that may favor malignancy (choice=Mild-moderate T2 hyperintensity)',
            '7.13 Features that may favor malignancy (choice=Restricted diffusion)',
            '7.13 Features that may favor malignancy (choice=Distinctive rim*)',
            '7.13 Features that may favor malignancy (choice=Corona enhancement*)',
            '7.13 Features that may favor malignancy (choice=Mosaic architecture*)',
            '7.13 Features that may favor malignancy (choice=Nodule-in-nodule architecture*)',
            '7.13 Features that may favor malignancy (choice=Intra-lesional fat*)',
            '7.13 Features that may favor malignancy (choice=Lesional iron sparing)',
            '7.13 Features that may favor malignancy (choice=Lesional fat sparing)',
            '7.13 Features that may favor malignancy (choice=Blood products)',
            '7.13 Features that may favor malignancy (choice=Diameter increase less than threshold growth)',
            '7.14 Features that may favor benignity (choice=Homogeneous marked T2 hyperintensity)',
            '7.14 Features that may favor benignity (choice=Homogeneous marked T2 or T2* hypointensity)',
            '7.14 Features that may favor benignity (choice=Hepatobiliary phase isointensity)',
            '7.14 Features that may favor benignity (choice=Undistorted vessels)',
            '7.14 Features that may favor benignity (choice=Parallels blood pool enhancement)',
            '7.14 Features that may favor benignity (choice=Diameter reduction)',
            '7.14 Features that may favor benignity (choice=Diameter stability ≥ 2 years)',
            '7.15 Tie-breaking Rules Applied',
            '7.16 CT / MRI LI-RADS Classification'
        ],
        'F8: Histology':[
            '8.1 Date',
            '8.2 Nodule',
            '8.3 Nodule Longest Diameter, mm',
            '8.4 Location (Couinaud segment)',
            '8.5 Nodule Tissue Sampling',
            '8.6 Needle Size (G)',
            '8.7 Results - Tumor Type',
            '8.8 Specify',
            '8.9 Edmondson and Steiner Grade',
            '8.10 WHO Grade',
            '8.10 Adjacent Liver Tissue Sampling',
            '8.11 Needle Size (G)',
            '8.12 Results',
            '8.13 Specify'
        ],
        'FINAL DIAGNOSIS':[
            'Nodule',
            'Final diagnosis source',
            'FINAL DIAGNOSIS',
            'Final notes'
        ]
    }

    records = dict()
    for row_idx in df.index:
        pat_id = df.loc[row_idx,'1.1 Patient ID']
        instrument = df.loc[row_idx,'Repeat Instrument']
        instance = df.loc[row_idx,'Repeat Instance']
        if pat_id not in records:
            records[pat_id] = dict()
        if instance not in records[pat_id]:
            records[pat_id][instance] = dict()
        cols = map_columns[instrument]
        records[pat_id][instance][instrument] = df.loc[row_idx,cols].to_dict()

    for pat_id in records.keys():
        sorted_instances = sorted(records[pat_id].keys())
        if len(sorted_instances) == 1:
            continue
        for instance in sorted_instances[1:]:
            if 'demographics_and_labs' not in records[pat_id][instance]:
                records[pat_id][instance]['demographics_and_labs'] = records[pat_id][1]['demographics_and_labs'].copy()
            if 'F3: Ultrasound Imaging' not in records[pat_id][instance]:
                records[pat_id][instance]['F3: Ultrasound Imaging'] = records[pat_id][1]['F3: Ultrasound Imaging'].copy()

    records_list = []
    for pat_id in records:
        for instance in records[pat_id].keys():
            temp = {
                '1.1 Patient ID': pat_id,
                'Repeat Instance': instance
            }
            for instrument in records[pat_id][instance].keys():
                temp.update(records[pat_id][instance][instrument])
            records_list.append(temp)

    df = pd.DataFrame.from_records(records_list)

    # number of patients and nodules
    #print('Original:')
    #print('Number of patients:', df['1.1 Patient ID'].nunique())
    #print('Number of nodules:', df.shape[0])

    df = df.loc[df['6.25 CEUS LI-RADS'].isin(['LR-3','LR-4'])].reset_index(drop=True)
    #print('After filtering for CEUS LI-RADS 3 and 4:')
    #print('Number of patients:', df['1.1 Patient ID'].nunique())
    #print('Number of nodules:', df.shape[0])

    df = df.loc[df['FINAL DIAGNOSIS'].isin(['HCC','BENIGN','OTHER MALIGNANCY'])].reset_index(drop=True)
    #print('After filtering for HCC, BENIGN, and OTHER MALIGNANCY:')
    #print('Number of patients:', df['1.1 Patient ID'].nunique())
    #print('Number of nodules:', df.shape[0])

    #print(pd.crosstab(df['6.25 CEUS LI-RADS'],df['FINAL DIAGNOSIS'], dropna=False, margins=True))

    df['age_years'] = df['6.1 Date'].apply(extract_year_from_str_date) - df['1.6 Year of Birth'].astype('int')
    df['BMI'] = df['2.1 Weight (kilograms)'] / ((df['2.2 Height (cm)']/100)**2)
    df.loc[:,'2.4 Personal History of treated HCC'] = df['2.4 Personal History of treated HCC'].fillna(value='No')
    df.loc[:,'2.11 Encephalopathy'] = df['2.11 Encephalopathy'].fillna(value='No Encephalopathy')
    df.loc[:,'2.12 Ascites'] = df['2.12 Ascites'].fillna(value='Absent')
    df.loc[(df['2.7 Cirrhosis']=='Yes') & \
        (df['2.14 MELD Score'].isna()|df['2.14 MELD Score'].isin(
            ['N/A','NA','not computable'])), '2.14 MELD Score'] = None
    df.loc[(df['2.7 Cirrhosis']=='No') & \
        (df['2.14 MELD Score'].isna()|df['2.14 MELD Score'].isin(
            ['N/A','NA','not computable'])), '2.14 MELD Score'] = 0
    df.loc[:,'2.14 MELD Score'] = df['2.14 MELD Score'].astype('float')
    df.loc[(df['2.7 Cirrhosis']=='Yes') & \
        (df['2.15 Child Pugh Score'].isna()|df['2.15 Child Pugh Score'].isin(
            ['N/A','NA','not computable'])), '2.15 Child Pugh Score'] = 'A'
    df.loc[(df['2.7 Cirrhosis']=='No') & \
        (df['2.15 Child Pugh Score'].isna()|df['2.15 Child Pugh Score'].isin(
            ['N/A','NA','not computable'])), '2.15 Child Pugh Score'] = 'No Cirrhosis'
    df.loc[df['5.1 Albumin, g/dL']>=10,'5.1 Albumin, g/dL'] = df.loc[df['5.1 Albumin, g/dL']>=10]['5.1 Albumin, g/dL'] / 10

    df.loc[df['5.6 Creatinine, mg/dL']=='not available','5.6 Creatinine, mg/dL'] = None
    df.loc[:,'5.6 Creatinine, mg/dL'] = df['5.6 Creatinine, mg/dL'].astype('float')
    df.loc[df['5.7 Sodium, mEq/L']=='na','5.7 Sodium, mEq/L'] = None
    df.loc[:,'5.7 Sodium, mEq/L'] = df['5.7 Sodium, mEq/L'].astype('float')
    df.loc[df['5.8 INR']=='1.29 (May 2018)','5.8 INR'] = 1.29
    df.loc[:,'5.8 INR'] = df['5.8 INR'].astype('float')
    df.loc[df['6.6 Nodule Longest Diameter, mm']=='26 mm','6.6 Nodule Longest Diameter, mm'] = 26
    df.loc[df['6.6 Nodule Longest Diameter, mm']=='19mm','6.6 Nodule Longest Diameter, mm'] = 19
    df.loc[:,'6.6 Nodule Longest Diameter, mm'] = df['6.6 Nodule Longest Diameter, mm'].astype('float')
    df.loc[:,'6.7 Location (Couinaud segment)'] = df['6.7 Location (Couinaud segment)'].apply(map_segment_to_str)
    df.loc[:,'6.19 Washout'] = df['6.19 Washout'].fillna(value='Absent')
    df.loc[(df['6.19 Washout']=='Absent') & \
        (df['6.20 Timing of Washout Onset, sec'].isna()), '6.20 Timing of Washout Onset, sec'] = 5000
    df.loc[(df['6.19 Washout']=='Absent') & \
        (df['6.21 Washout Degree'].isna()), '6.21 Washout Degree'] = 'no washout'

    map_Dx_to_GT = {
        'HCC': 1,
        'OTHER MALIGNANCY': 1,
        'BENIGN': 0
    }
    df['GT_benign(0)_malignant(1)'] = df['FINAL DIAGNOSIS'].apply(lambda x:map_Dx_to_GT[x])
    
    feature_cols = [
        'age_years',
        '1.7 Gender',
        '1.8 Race',
        '1.9 Ethnicity',
        'BMI',
        '2.3 Personal_history of HCC',
        '2.4 Personal History of treated HCC',
        '2.5 Liver Disease Etiology (choice=Alcohol)',
        '2.5 Liver Disease Etiology (choice=NASH)',
        '2.5 Liver Disease Etiology (choice=Hepatitis B)',
        '2.5 Liver Disease Etiology (choice=Hepatitis C)',
        '2.5 Liver Disease Etiology (choice=Other)',
        '2.7 Cirrhosis',
        '2.11 Encephalopathy',
        '2.12 Ascites',
        '2.14 MELD Score',
        '2.15 Child Pugh Score',
        '5.1 Albumin, g/dL',
        '5.2 AFP, ng/ml',
        '5.3 ALT, units',
        '5.4 AST, units',
        '5.5 Bilirubin (Total), mg/dL',
        '5.6 Creatinine, mg/dL',
        '5.7 Sodium, mEq/L',
        '5.8 INR',
        '6.6 Nodule Longest Diameter, mm',
        '6.7 Location (Couinaud segment)',
        '6.9 Nodule Echogenicity',
        '6.10 Nodule Margin',
        '6.17 Arterial Phase Enhancement',
        '6.18 Arterial Phase Enhancement Distribution',
        '6.19 Washout',
        '6.20 Timing of Washout Onset, sec',
        '6.21 Washout Degree',
        '6.25 CEUS LI-RADS'
    ]

    GT_col = 'GT_benign(0)_malignant(1)'

    feature_df = df[feature_cols]
    labels = df[GT_col]

    feature_df = convert_categorical_to_onehot(feature_df)

    feature_df = feature_df.drop(columns = ['1.7 Gender_Female',
                                            '2.3 Personal_history of HCC_No',
                                            '2.4 Personal History of treated HCC_No',
                                            '2.5 Liver Disease Etiology (choice=Alcohol)_Unchecked',
                                            '2.5 Liver Disease Etiology (choice=NASH)_Unchecked',
                                            '2.5 Liver Disease Etiology (choice=Hepatitis B)_Unchecked',
                                            '2.5 Liver Disease Etiology (choice=Hepatitis C)_Unchecked',
                                            '2.5 Liver Disease Etiology (choice=Other)_Unchecked',
                                            '2.7 Cirrhosis_No',
                                            '2.11 Encephalopathy_No Encephalopathy',
                                            '6.19 Washout_Absent',
                                            '6.25 CEUS LI-RADS_LR-3'])
    
    # remove leading number from column names
    feature_df.columns = feature_df.columns.str.replace(r'^\d+\.\d+\s', '', regex=True)

    # remove trailing "_Checked" from column names
    feature_df.columns = feature_df.columns.str.replace(r'_Checked$', '', regex=True)
    
    # split the data into train and test sets, by patient
    # use StratifiedGroupKFold to ensure that the same patient does not appear in both train and test sets
    sgkf = StratifiedGroupKFold(n_splits=4, random_state=random_state, shuffle=True)
    for train_index, test_index in sgkf.split(feature_df, labels, groups=df['1.1 Patient ID']):
        nodules_train, nodules_test = feature_df.iloc[train_index], feature_df.iloc[test_index]
        labels_train, labels_test = labels.iloc[train_index], labels.iloc[test_index]
        break

    df.loc[nodules_train.index, 'allocated_set'] = 'train'
    df.loc[nodules_test.index, 'allocated_set'] = 'test'
    
    return (nodules_train, 
            nodules_test, 
            labels_train, 
            labels_test, 
            feature_df.columns, 
            df.loc[nodules_train.index]['1.1 Patient ID'], 
            df.loc[nodules_test.index]['1.1 Patient ID'], 
            df[['1.1 Patient ID']+feature_cols+[GT_col]+['allocated_set']+confirmatory_tests_cols])


def refit_strategy(cv_results):
    """
    Select the model to refit on the whole dataset.
    Steps:
    1. Keep only models with mean specificity >= 0.9 (mean across all CV folds).
    If no model achieved 0.9 specificity, pick the models achieving highest specificity possible.
    2. Then, find the highest mean sensitivity value among the models from Step 1
    3. keep only models achieving the highest sensitivity (maybe more than one model)
    4. Among the models from Step 3, select the model achieving the highest specificity
    Returns:
    best_index : int
        The index of the best estimator as it appears in `cv_results`.
    """
    cv_df = pd.DataFrame(cv_results)

    if (cv_df['mean_test_specificity']>=spec_threshold).sum() > 0:
        #keep only rows achieving the specificity threshold
        cv_df = cv_df.loc[cv_df['mean_test_specificity']>=spec_threshold]
    else:
        #If no model achieved 0.9 specificity, pick the models achiving highest specificity possible.
        max_spec = cv_df['mean_test_specificity'].max()
        cv_df = cv_df.loc[cv_df['mean_test_specificity']==max_spec]
    
    #find the highest mean sensitivity value among the models from Step 1
    max_sens = cv_df['mean_test_sensitivity'].max()
    #keep only models achieving the highest sensitivity (maybe more than one model)
    cv_df = cv_df.loc[cv_df['mean_test_sensitivity']==max_sens]
    #select the model achieving the highest specificity
    best_index = cv_df['mean_test_specificity'].idxmax()
    return best_index


def cross_validate(nodules_train, labels_train, export_results_path, patients_train):
    model = HistGradientBoostingClassifier(random_state=random_state)

    param_grid = {
        "learning_rate": [0.01, 0.03, 0.1, 0.3],
        "max_iter": [20, 50, 100, 200, 400],
        "max_depth": [None, 5, 10],
        "max_features": [0.1, 0.5, 1.0],
        "max_leaf_nodes": [None, 5, 10, 20],
        "min_samples_leaf": [1, 5, 10],
        "class_weight": [{0:5.0,1:1.0}]
    }

    cv_scoring = {
      "roc_auc": make_scorer(roc_auc_score),
      "balanced_accuracy": make_scorer(balanced_accuracy_score),
      "accuracy": make_scorer(accuracy_score),
      "sensitivity": make_scorer(recall_score),
      "specificity": make_scorer(recall_score, pos_label=0),
        "PPV": make_scorer(precision_score),
        "NPV": make_scorer(precision_score, pos_label=0)
    }

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    grid = GridSearchCV(model, 
                        param_grid, 
                        cv=cv,
                        scoring=cv_scoring, 
                        refit=refit_strategy,
                        n_jobs=-1, 
                        verbose=1, 
                        return_train_score=True)
    
    cv_results = grid.fit(nodules_train, labels_train, groups=patients_train)
    results_chart = pd.DataFrame(cv_results.cv_results_)
    results_chart.to_excel(export_results_path)
    
    return grid


# main codes

In [ ]:
nodules_train, nodules_test, labels_train, labels_test, feature_columns, patients_train, patients_test, df = load_data(DATA_PATH)

df.to_csv(export_raw_final_analysis_group_path, index=True)

# Patient-level descriptives

In [ ]:
descriptive_analyses(df.drop_duplicates(subset=['1.1 Patient ID']))

# Nodule-level descriptives

In [ ]:
descriptive_analyses(df)

# Grid search

In [ ]:
grid = cross_validate(nodules_train, labels_train, export_results_path, patients_train)
print(grid.best_params_)
print('sensitivity:', grid.cv_results_['mean_test_sensitivity'][grid.best_index_],
     'SD:', grid.cv_results_['std_test_sensitivity'][grid.best_index_])
print('specificity:', grid.cv_results_['mean_test_specificity'][grid.best_index_],
     'SD:', grid.cv_results_['std_test_specificity'][grid.best_index_])
print('PPV:', grid.cv_results_['mean_test_PPV'][grid.best_index_],
     'SD:', grid.cv_results_['std_test_PPV'][grid.best_index_])
print('NPV:', grid.cv_results_['mean_test_NPV'][grid.best_index_],
     'SD:', grid.cv_results_['std_test_NPV'][grid.best_index_])

# feature importance using SHAP

In [ ]:
import shap

explainer = shap.Explainer(grid.best_estimator_)
shap_values = explainer(pd.concat([nodules_train, nodules_test], axis=0))

shap.plots.beeswarm(shap_values, max_display=20)

# performance on the test set

In [ ]:
pred_test = grid.best_estimator_.predict(nodules_test)
print('test set')
print('sensitivity:', recall_score(labels_test, pred_test))
print('specificity:', recall_score(labels_test, pred_test, pos_label=0))
print('PPV:', precision_score(labels_test, pred_test))
print('NPV:', precision_score(labels_test, pred_test, pos_label=0))

In [11]:

predictions_df = pd.DataFrame({'1.1 Patient ID': patients_test, 'GT_benign(0)_malignant(1)': labels_test, 'prediction': pred_test})
predictions_df = pd.concat([predictions_df, nodules_test], axis=1)
predictions_df.to_csv(export_raw_predictions_path, index=False)


# Bootstrap test set

In [ ]:
from tqdm import tqdm

np.random.seed(random_state)
iterations = 10000

n_test = nodules_test.shape[0]

results = {
    'sensitivity':[],
    'specificity':[],
    'PPV':[],
    'NPV':[]
}

for i in tqdm(range(iterations)):
    indices = np.random.choice(nodules_test.index, size=n_test, replace=True)
    bootstrap_nodules_test = nodules_test.loc[indices]
    bootstrap_labels_test = labels_test.loc[indices]
    bootstrap_pred_test = grid.best_estimator_.predict(bootstrap_nodules_test)
    results['sensitivity'].append(recall_score(bootstrap_labels_test, bootstrap_pred_test))
    results['specificity'].append(recall_score(bootstrap_labels_test, bootstrap_pred_test, pos_label=0))
    results['PPV'].append(precision_score(bootstrap_labels_test, bootstrap_pred_test))
    results['NPV'].append(precision_score(bootstrap_labels_test, bootstrap_pred_test, pos_label=0))

print('95% CI')
for metric in results.keys():
    print('{0}: ({1:.3f}, {2:.3f})'.format(metric,
                                          np.quantile(results[metric],0.025),
                                          np.quantile(results[metric],0.975)))

# Bootstrap dataset split

In [ ]:
iterations = 1000

test_results = {
    'sensitivity':[],
    'specificity':[],
    'PPV':[],
    'NPV':[]
}

for i in tqdm(range(iterations)):
    random_state = i
    bootstrap_nodules_train, bootstrap_nodules_test, bootstrap_labels_train, bootstrap_labels_test, feature_columns, bootstrap_patients_train, bootstrap_patients_test, bootstrap_df = load_data(DATA_PATH)
    
    best_params = grid.best_params_
        
    bootstrap_model = HistGradientBoostingClassifier(random_state=random_state,
                                                    **best_params)

    bootstrap_model.fit(bootstrap_nodules_train, bootstrap_labels_train)
    bootstrap_pred_test = bootstrap_model.predict(bootstrap_nodules_test)

    test_results['sensitivity'].append(recall_score(bootstrap_labels_test, bootstrap_pred_test))
    test_results['specificity'].append(recall_score(bootstrap_labels_test, bootstrap_pred_test, pos_label=0))
    test_results['PPV'].append(precision_score(bootstrap_labels_test, bootstrap_pred_test))
    test_results['NPV'].append(precision_score(bootstrap_labels_test, bootstrap_pred_test, pos_label=0))

for metric in test_results.keys():
    print('{0}: {1:.3f} ({2:.3f}, {3:.3f})'.format(metric,
                                                  np.mean(test_results[metric]), 
                                                  np.quantile(test_results[metric],0.025),
                                                  np.quantile(test_results[metric],0.975)))